In [1]:
from ga import Genetic
from IO import DataWarehouse
from pathlib import Path
import os

# Helper function

In [2]:
def write_output(solution: list[dict], file_name: str, data) -> None:
    
    output_file_multiple_sets = file_name
    with open(output_file_multiple_sets, "w") as file:
        for s in solution:
            sorted_elements = sorted(s)
            line = " ".join(map(str, sorted_elements)) + f" #UTIL: {Genetic.evaluation(s, data)}"
            file.write(line + "\n")

In [3]:
def write_output(solution: list[dict], file_name: str, data) -> None:
    
    output_file_multiple_sets = file_name
    with open(output_file_multiple_sets, "w") as file:
        for s in solution:
            sorted_elements = sorted(s)
            line = " ".join(map(str, sorted_elements)) + f" #UTIL: {Genetic.evaluation(s, data)}"
            file.write(line + "\n")

# Run through all data

In [ ]:
folder_path = Path("data")
all_files = folder_path.glob("*")

ga = Genetic(
    number_of_population=25,
    m=10,
    quantity_of_elite=15,
    k_tournament=5,
    number_population_s=5,
    stop_criteria_loop=100,
    time_limit=100,
)

for file_path in all_files:
    data_name = os.path.splitext(os.path.basename(file_path))[0]
    dir_output = os.path.join("outputs", data_name)
    print(data_name)
    if not os.path.exists(dir_output):
        os.makedirs(dir_output)

    with file_path.open("r", encoding="utf-8") as file:
        data = DataWarehouse(file.read())
        for i in range(0, 5):
            solution = ga.solve(data)
            write_output(
                solution, f"{dir_output}/sol_{i}", data
            )

accidents_negative
